In [17]:
import keras
from keras.layers import Input, Dense, Lambda, Flatten
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras.backend as K
import tensorflow as tf

from models import vgg
from utils.angles import deg2bit, bit2deg
from utils.losses import maad_from_deg 
from utils.losses import von_mises_log_likelihood_tf, von_mises_log_likelihood_np
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

In [57]:
np.exp(-1.073867)

0.34168466480059195

In [58]:
np.exp( -1.037539)

0.35432560516251921

In [2]:
xtr, ytr_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True)
image_height, image_width = xtr.shape[1], xtr.shape[2]
ytr_bit = deg2bit(ytr_deg)
yte_bit = deg2bit(yte_deg)

In [59]:
X = Input(shape=[50,50,3])

vgg_x = vgg.vgg_model(final_layer=False,
                      image_height=50,
                      image_width=50)(X)

mu = Lambda(lambda x: K.l2_normalize(x, axis=1))(Dense(2)(vgg_x))

kappa_pred = tf.ones([tf.shape(mu)[0],1])

# kappa_pred = Lambda(lambda x: K.abs(x))(Dense(1)(vgg_x))

def _von_mises_neg_log_likelihood_keras(y_true, y_pred):
    return -von_mises_log_likelihood_tf(y_true, y_pred, kappa_pred, input_type='biternion')

biternion_vgg_prob = Model(X, mu)
# kappa_model = Model(X, concatenate([kappa_pred, kappa_pred]))

optimizer = keras.optimizers.Adam(lr=0.0001)

biternion_vgg_prob.compile(loss=_von_mises_neg_log_likelihood_keras,
                           metrics=['cosine'],
                           optimizer=optimizer)

In [71]:
vals

array([ 0. ,  0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,
        1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,
        2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,
        3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,
        4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,
        5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,
        6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,
        7.7,  7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,
        8.8,  8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,
        9.9])

In [73]:
res = np.zeros(vals.shape)

In [72]:
vals>0

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,  True], dtype=bool)

In [44]:
biternion_vgg_prob.fit(x=xtr , y=ytr_bit, batch_size=10, validation_split=0.1, epochs=20)

Train on 7002 samples, validate on 778 samples
Epoch 1/20
3300/7002 [=============>................] - ETA: 43s - loss: 1.7926 - cosine_proximity: -0.1743

KeyboardInterrupt: 

In [22]:
yte_preds_bit = biternion_vgg_prob.predict(xte)
yte_preds_deg = deg2bit(yte_preds_bit)

In [36]:
von_mises_log_likelihood_np(yte_bit, yte_bit, np.ones([yte_bit.shape[0],1])*200, input_type='biternion')

1.7295935793671882

In [25]:
np.ones(y)

array([[ -6.96785189e-03],
       [ -8.87782406e-03],
       [  8.30960926e-03],
       [ -3.83389555e-03],
       [ -2.28828844e-03],
       [ -1.65289012e-03],
       [ -1.98708498e-03],
       [  3.36911413e-03],
       [  7.34762289e-05],
       [  5.86505327e-03],
       [ -2.01713666e-03],
       [ -3.77813657e-03],
       [ -2.39501614e-03],
       [ -1.15487967e-02],
       [ -6.64312905e-03],
       [ -1.27037987e-04],
       [  3.89955007e-04],
       [  2.93105701e-03],
       [ -8.20408296e-03],
       [ -4.65073809e-03],
       [  9.33538191e-04],
       [  7.51849823e-03],
       [  1.90972257e-03],
       [  2.17655953e-03],
       [  1.16547942e-03],
       [ -5.35069592e-03],
       [ -5.66245336e-03],
       [ -8.19366612e-03],
       [ -6.13136357e-03],
       [ -7.44347461e-03],
       [ -5.99281956e-03],
       [ -3.90916876e-03],
       [ -1.32103087e-02],
       [ -8.88172165e-03],
       [ -1.40123507e-02],
       [  6.96688006e-03],
       [  2.24007815e-02],
 

In [10]:
kappa_model = Model(X, kappa_pred)

In [16]:
kappa_te_preds = kappa_model.predict(xte)

In [13]:
loss = maad_from_deg(yte_preds, yte_deg)

mean_loss = np.mean(loss)
std_loss = np.std(loss)

print("MAAD error (test) : %f ± %f" % (mean_loss, std_loss))

MAAD error (test) : 79.814962 ± 53.066745
